#Collega Google Drive a Colab

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#Install FAISS

In [ ]:
!pip install -q condacolab
import condacolab
condacolab.install()

!conda --version

⏬ Downloading https://github.com/conda-forge/miniforge/releases/download/23.11.0-0/Mambaforge-23.11.0-0-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:08
🔁 Restarting kernel...
conda 23.11.0


In [ ]:
# GPU(+CPU) version
! conda install -c pytorch -c nvidia faiss-gpu=1.8.0

Channels:
 - pytorch
 - nvidia
 - conda-forge
Platform: linux-64
Solving environment: / - \ done


==> WARNING: A newer version of conda exists. <==
    current version: 23.11.0
    latest version: 24.5.0

Please update conda by running

    $ conda update -n base -c conda-forge conda



## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - faiss-gpu=1.8.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    _openmp_mutex-4.5          |       2_kmp_llvm           6 KB  conda-forge
    ca-certificates-2024.7.4   |       hbcca054_0         151 KB  conda-forge
    certifi-2024.6.2           |     pyhd8ed1ab_0         157 KB  conda-forge
    cuda-cudart-11.8.89        |                0         197 KB  nvidia
    faiss-gpu-1.8.0            |py3.10_hedc54c9_0_cuda11.4.4         5.0 MB  pytorch
    libblas-3.9.0              |22_linux64_openblas          14 

#Clone repository and requirements

In [ ]:
!git clone https://github.com/omarfoq/knn-per.git

Cloning into 'knn-per'...
remote: Enumerating objects: 116, done.
remote: Counting objects: 100% (116/116), done.
remote: Compressing objects: 100% (74/74), done.
remote: Total 116 (delta 44), reused 107 (delta 38), pack-reused 0
Receiving objects: 100% (116/116), 71.48 KiB | 14.30 MiB/s, done.
Resolving deltas: 100% (44/44), done.


In [ ]:
%cd knn-per/
!pip install -r requirements.txt

[Errno 2] No such file or directory: 'knn-per/'
/content/knn-per
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.2/162.2 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 111.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 779.1/779.1 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 98.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 80.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 46.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 33.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124

#Generate data

In [ ]:
%cd /content/knn-per/data/cifar100
!python generate_data.py \
    --n_tasks 100 \
    --n_components -1 \
    --s_frac 1.0 \
    --test_tasks_frac 0.0 \
    --seed 12345
%cd ../..
#--by_labels_split \
#--alpha 0.3 \

/content/knn-per/data/cifar100
100% 169001437/169001437 [00:13<00:00, 12724180.90it/s]
Extracting raw_data/cifar-100-python.tar.gz to raw_data/
/content/knn-per


**aggiungere modello in models.py e utils.utils.py per modificare il modello per il training (elif name == "cifar100":)**

in client.py si calcola shared rappresentation

#Training

In [ ]:
!python /content/knn-per/train.py cifar100 \
        --aggregator_type centralized \
        --sampling_rate 0.1 \
        --model_name ModifiedLeNet5 \
        --n_rounds 2000 \
        --local_steps 4 \
        --bz 64 \
        --lr 0.01 \
        --lr_scheduler constant \
        --log_freq 10 \
        --device cuda \
        --optimizer sgd \
        --seed 1234 \
        --logs_dir ./logs \
        --chkpts_dir ./chkpts/cifar10_fedavg \
        --verbose 1

# --model_name ModifiedLeNet5
# --local_steps 4

python3: can't open file '/content/knn-per/train.py': [Errno 2] No such file or directory


#Salva i risultati su Google Drive

In [ ]:
from google.colab import drive
import os
import shutil

# Monta Google Drive
drive.mount('/content/drive')

# Percorsi
source_path = '/content/knn-per'
dest_path = '/content/drive/MyDrive/knn-per'

# Crea la cartella di destinazione se non esiste
os.makedirs(dest_path, exist_ok=True)

# Copia la directory dalla sorgente alla destinazione
if os.path.exists(source_path):
    shutil.copytree(source_path, dest_path, dirs_exist_ok=True)
else:
    print(f"La directory di origine '{source_path}' non esiste.")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
